In [2]:
import sys
sys.path.insert(0,'/exp/rhuang/meta/icefall/egs/librispeech/ASR/conformer_ctc2_noseg')
sys.path.insert(0,'/exp/rhuang/meta/k2/k2/python')
sys.path.insert(0,'/exp/rhuang/meta/k2/temp.linux-x86_64-cpython-310/lib')
sys.path.insert(0,'/exp/rhuang/meta/icefall')

In [3]:
%cd /home/rhuang25/work/icefall/egs/librispeech/ASR
%pwd  #look at the current work dir

/scratch4/skhudan1/rhuang25/icefall/egs/librispeech/ASR


/home/rhuang25/.conda/envs/aligner5/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


'/scratch4/skhudan1/rhuang25/icefall/egs/librispeech/ASR'

In [4]:
import argparse
import copy
import logging
import warnings
from pathlib import Path
from shutil import copyfile
from typing import Any, Dict, Optional, Tuple, Union
from collections import defaultdict

import k2
import optim
import sentencepiece as spm
import torch
import torch.multiprocessing as mp
import torch.nn as nn
from asr_datamodule import LibriSpeechAsrDataModule
from decoder import Decoder
from joiner import Joiner
from lhotse.cut import Cut
from lhotse.dataset.sampling.base import CutSampler
from lhotse.utils import fix_random_seed
from model import AsrModel
from optim import Eden, ScaledAdam
from scaling import ScheduledFloat
from subsampling import Conv2dSubsampling
from torch import Tensor
from torch.cuda.amp import GradScaler
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.tensorboard import SummaryWriter
from zipformer import Zipformer2

from icefall import diagnostics
from icefall.checkpoint import load_checkpoint, remove_checkpoints
from icefall.checkpoint import save_checkpoint as save_checkpoint_impl
from icefall.checkpoint import (
    save_checkpoint_with_global_batch_idx,
    update_averaged_model,
)
from icefall.dist import cleanup_dist, setup_dist
from icefall.env import get_env_info
from icefall.hooks import register_inf_check_hooks
from icefall.utils import (
    AttributeDict,
    MetricsTracker,
    get_parameter_groups_with_lrs,
    setup_logger,
    str2bool,
)

import psutil
try:
    from tqdm_loggable.auto import tqdm
except:
    from tqdm import tqdm
from train_concat_libri import (
    get_parser, 
    add_model_arguments,
    get_params,
    get_model,
    load_checkpoint_if_available,
)
from ctc_decode import get_decoding_params 
from icefall.decode import get_lattice, one_best_decoding
from icefall.utils import get_alignments, get_texts, get_texts_with_timestamp

/home/rhuang25/.conda/envs/aligner5/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
parser = get_parser()
# add_model_arguments(parser)
LibriSpeechAsrDataModule.add_arguments(parser)
args = parser.parse_args(args=[])

# args.exp_dir = "/home/rhuang25/work/icefall/egs/librispeech/ASR/zipformer_no_seg/exp-test/"
# args.bpe_model = "/home/rhuang25/work/icefall/egs/librispeech/ASR/data/lang_bpe_500/bpe.model"
# args.epoch = 1
# args.avg = 1

# args.exp_dir = "/home/rhuang25/work/icefall/egs/librispeech/ASR/zipformer/exp-ctc-rnnt"
# args.bpe_model = "/home/rhuang25/work/icefall/egs/librispeech/ASR/data/lang_bpe_500/bpe.model"
# args.epoch = 40
# args.avg = 1

args.exp_dir = "/home/rhuang25/work/icefall/egs/librispeech/ASR/zipformer_zero_grad/exp-test/"
args.bpe_model = "/home/rhuang25/work/icefall/egs/librispeech/ASR/data/lang_bpe_500/bpe.model"
args.epoch = 2
args.avg = 1

args.manifest_dir = "/home/rhuang25/work/icefall/egs/librispeech/ASR/data/fbank/"
args.max_duration = 30
args.world_size = 1
args.start_epoch = 2
args.full_libri = False
args.use_transducer = False
args.use_ctc = True
args.ctc_loss_scale = 1.0
args.ctc_beam_size = 4

args.exp_dir = Path(args.exp_dir)
args.manifest_dir = Path(args.manifest_dir)
params = get_params()
params.update(get_decoding_params())
params.update(vars(args))

fix_random_seed(params.seed)

device = torch.device("cpu")

sp = spm.SentencePieceProcessor()
sp.load(params.bpe_model)

params.blank_id = sp.piece_to_id("<blk>")
params.unk_id = sp.piece_to_id("<unk>")
params.vocab_size = sp.get_piece_size()

if not params.use_transducer:
    params.ctc_loss_scale = 1.0

In [6]:
print(params)

{'best_train_loss': inf, 'best_valid_loss': inf, 'best_train_epoch': -1, 'best_valid_epoch': -1, 'batch_idx_train': 0, 'log_interval': 50, 'reset_interval': 200, 'valid_interval': 1000, 'feature_dim': 80, 'subsampling_factor': 4, 'warm_step': 2000, 'env_info': {'k2-version': '1.24.4', 'k2-build-type': 'Debug', 'k2-with-cuda': True, 'k2-git-sha1': '415fe1f446fffe1d9e7219b5033966294c0b430c', 'k2-git-date': 'Wed Dec 20 21:38:57 2023', 'lhotse-version': '1.20.0.dev+git.c6788492.clean', 'torch-version': '2.1.2', 'torch-cuda-available': True, 'torch-cuda-version': '11.8', 'python-version': '3.1', 'icefall-git-branch': 'noseg', 'icefall-git-sha1': '321a61a-dirty', 'icefall-git-date': 'Tue Feb 6 19:21:58 2024', 'icefall-path': '/scratch4/skhudan1/rhuang25/icefall', 'k2-path': '/scratch4/skhudan1/rhuang25/k2/k2/python/k2/__init__.py', 'lhotse-path': '/scratch4/skhudan1/rhuang25/lhotse/lhotse/__init__.py', 'hostname': 'gpu08', 'IP address': '172.20.30.8'}, 'frame_shift_ms': 10, 'search_beam': 20

In [7]:
# params = AttributeDict(
#     {'best_train_loss': float('inf'), 'best_valid_loss': float('inf'), 'best_train_epoch': -1, 'best_valid_epoch': -1, 'batch_idx_train': 0, 'log_interval': 50, 'reset_interval': 200, 'valid_interval': 3000, 'feature_dim': 80, 'subsampling_factor': 4, 'warm_step': 2000, 'env_info': {'k2-version': '1.24.4', 'k2-build-type': 'Debug', 'k2-with-cuda': True, 'k2-git-sha1': '415fe1f446fffe1d9e7219b5033966294c0b430c', 'k2-git-date': 'Wed Dec 20 21:38:57 2023', 'lhotse-version': '1.20.0.dev+git.c6788492.clean', 'torch-version': '2.1.2', 'torch-cuda-available': True, 'torch-cuda-version': '11.8', 'python-version': '3.1', 'icefall-git-branch': 'noseg', 'icefall-git-sha1': 'b0ecb7f-dirty', 'icefall-git-date': 'Thu Feb 1 03:39:11 2024', 'icefall-path': '/scratch4/skhudan1/rhuang25/icefall', 'k2-path': '/scratch4/skhudan1/rhuang25/k2/k2/python/k2/__init__.py', 'lhotse-path': '/scratch4/skhudan1/rhuang25/lhotse/lhotse/__init__.py', 'hostname': 'gpu02', 'IP address': '172.20.30.2'}, 'frame_shift_ms': 10, 'search_beam': 20, 'output_beam': 8, 'min_active_states': 30, 'max_active_states': 10000, 'use_double_scores': True, 'epoch': 1, 'iter': 0, 'avg': 1, 'use_averaged_model': False, 'exp_dir': Path('zipformer_no_seg/exp-test'), 'bpe_model': 'data/lang_bpe_500/bpe.model', 'lang_dir': Path('data/lang_bpe_500'), 'context_size': 2, 'decoding_method': '1best', 'num_paths': 100, 'nbest_scale': 1.0, 'hlg_scale': 0.6, 'lm_dir': Path('data/lm'), 'num_encoder_layers': '2,2,3,4,3,2', 'downsampling_factor': '1,2,4,8,4,2', 'feedforward_dim': '512,768,1024,1536,1024,768', 'num_heads': '4,4,4,8,4,4', 'encoder_dim': '192,256,384,512,384,256', 'query_head_dim': '32', 'value_head_dim': '12', 'pos_head_dim': '4', 'pos_dim': 48, 'encoder_unmasked_dim': '192,192,256,256,256,192', 'cnn_module_kernel': '31,31,15,15,15,31', 'decoder_dim': 512, 'joiner_dim': 512, 'causal': False, 'chunk_size': '16,32,64,-1', 'left_context_frames': '64,128,256,-1', 'use_transducer': False, 'use_ctc': True, 'full_libri': True, 'mini_libri': False, 'manifest_dir': Path('data/fbank'), 'max_duration': 500, 'bucketing_sampler': True, 'num_buckets': 30, 'concatenate_cuts': False, 'duration_factor': 1.0, 'gap': 1.0, 'on_the_fly_feats': False, 'shuffle': True, 'drop_last': True, 'return_cuts': True, 'num_workers': 2, 'enable_spec_aug': True, 'spec_aug_time_warp_factor': 80, 'enable_musan': True, 'input_strategy': 'PrecomputedFeatures', 'res_dir': Path('zipformer_no_seg/exp-test/1best'), 'suffix': 'epoch-1-avg-1', 'vocab_size': 500, 'blank_id': 0}
# )

In [8]:
model = get_model(params)

# load_checkpoint(f"{params.exp_dir}/epoch-{params.epoch}.pt", model)
model_avg = copy.deepcopy(model).to(torch.float64)

assert params.start_epoch > 0, params.start_epoch
checkpoints = load_checkpoint_if_available(
    params=params, model=model, model_avg=model_avg
)

num_param = sum([p.numel() for p in model.parameters()])
print(f"Number of model parameters: {num_param}")

Number of model parameters: 64250603


In [9]:
# _ = model.train()
_ = model.eval()

In [10]:
librispeech = LibriSpeechAsrDataModule(args)

train_cuts = librispeech.train_clean_100_cuts()
# if params.full_libri:
#     train_cuts += librispeech.train_clean_360_cuts()
#     train_cuts += librispeech.train_other_500_cuts()
# train_cuts = librispeech.train_other_500_cuts()

# valid_cuts = librispeech.dev_clean_cuts()
# valid_cuts += librispeech.dev_other_cuts()

# test_cuts = librispeech.test_clean_cuts()
# test_cuts += librispeech.test_other_cuts()
# test_cuts = test_cuts.to_eager()

In [11]:
c = train_cuts.sample(n_cuts=1)
# c = test_cuts['1320-122612-0011-2251']
c

MonoCut(id='4362-15663-0058-1436_sp1.1', start=0.0, duration=9.736375, channel=0, supervisions=[SupervisionSegment(id='4362-15663-0058_sp1.1', recording_id='4362-15663-0058_sp1.1', start=0.0, duration=9.736375, channel=0, text='SUGGESTING TO US THE IDEA OF AT ONCE SEEKING OUR SAFE CAMP IN THE TIMBER AND ABANDONING THE PURPOSE OF MAKING AN OBSERVATION OF THE BAROMETER AT THREE P M TWO HAVING ALREADY BEEN MADE', language='English', speaker='4362', gender=None, custom=None, alignment=None)], features=Features(type='kaldi-fbank', num_frames=974, num_features=80, frame_shift=0.01, sampling_rate=16000, start=0.0, duration=9.736375, storage_type='lilcom_chunky', storage_path='data/fbank/librispeech_feats_train-clean-100/feats-14.lca', storage_key='480287609,46822,44423', recording_id='None', channels=0), recording=Recording(id='4362-15663-0058_sp1.1', sources=[AudioSource(type='file', channels=[0], source='/home/rhuang25/work/icefall/egs/librispeech/ASR/download/LibriSpeech/train-clean-100/43

In [12]:
c.play_audio()

In [13]:
def get_uid_key(my_id):
    # /data/skhudan1/corpora/librispeech/CHAPTERS.TXT
    speaker_id, chapter_id, utterance_id, _ = my_id.split("-")
    speaker_id, chapter_id, utterance_id = int(speaker_id), int(chapter_id), int(utterance_id)
    return speaker_id, chapter_id, utterance_id


def convert_long_text_to_fst(items, sp):
    libri_long_text_sp = dict()
    for k, text in tqdm(items, mininterval=2, desc=f"libri_long_text"):
        libri_long_text_sp[k] = make_factor_transducer1(sp.encode(text, out_type=int), return_str=True, blank_penalty=0)
        # libri_long_text_sp[k] = make_factor_transducer2(sp.encode(text, out_type=int), return_str=True, blank_penalty=-12)
    return libri_long_text_sp


def get_long_text(cuts, sp=None, make_fst=False):
    logging.info(f"Getting long text from cuts ... ")  # len(cuts) = {len(cuts)}
    cuts_by_recoding = defaultdict(list)
    for i, c in enumerate(cuts):  # tqdm(cuts, miniters=1000, total=None):
        # if "_sp" in c.id:
        #     continue
        cuts_by_recoding[tuple(get_uid_key(c.id)[:2])].append(c)

        # if i % 1e4 == 0:
        #     print(f"progress: {i}")

    libri_long_text = dict()
    for k, v in cuts_by_recoding.items():
        v.sort(key = lambda x: get_uid_key(x.id)[-1])
        text = " ".join([c.supervisions[0].text for c in v])
        libri_long_text[k] = text
    
    if sp is None:
        return libri_long_text

    logging.info(f"Converting long text to fst ... ")

    if not make_fst:
        libri_long_text_sp = dict()
        for k, text in libri_long_text.items():
            libri_long_text_sp[k] = sp.encode(text, out_type=int)
    else:
        results = [0]
        libri_long_text_sp = convert_long_text_to_fst(
            libri_long_text.items(),
            sp,
        )
        for k, v in tqdm(libri_long_text_sp.items()):
            libri_long_text_sp[k] = k2.Fsa.from_str(v, acceptor=False)
    
    ram_info = psutil.virtual_memory()
    ram_used_mb = ram_info.used / (1024 ** 2)  # Convert bytes to megabytes
    ram_total_mb = ram_info.total / (1024 ** 2)  # Convert bytes to megabytes
    ram_usage = ram_info.percent
    logging.info(f"Current RAM Usage: {ram_used_mb:.2f} MB out of {ram_total_mb:.2f} MB ({ram_usage}%)")

    return libri_long_text_sp


def make_factor_transducer1(word_id_list, return_str=False, blank_penalty=0):
    # This is the original, simplest factor transducer for a "linear" fst

    fst_graph = k2.ctc_graph([word_id_list], modified=False, device='cpu')[0]

    c_str = k2.to_str_simple(fst_graph)
    arcs = c_str.strip().split("\n")
    arcs = [x.strip() for x in arcs if len(x.strip()) > 0]
    final_state = int(arcs[-1])
    
    arcs = arcs[:-1]
    arcs = [tuple(map(int, a.split())) for a in arcs]
    # ss, ee, l1, l2, w = arc

    non_eps_nodes = set((arc[1], arc[3]) for arc in arcs if arc[3] > -1)   # if this node has a non-eps in-coming arc
    arcs += [(0, n, l, l, 0) for n, l in non_eps_nodes if n > 1]

    arcs += [(n, final_state, -1, -1, 0) for n in range(1, final_state - 2)]

    new_arcs = arcs
    new_arcs.append([final_state])

    new_arcs = sorted(new_arcs, key=lambda arc: arc[0])
    new_arcs = [[str(i) for i in arc] for arc in new_arcs]
    new_arcs = [" ".join(arc) for arc in new_arcs]
    new_arcs = "\n".join(new_arcs)

    if return_str:
        return new_arcs
    else:
        fst = k2.Fsa.from_str(new_arcs, acceptor=False)
        return fst


def make_factor_transducer2(word_id_list, return_str=False, blank_penalty=-1):
    # This is the factor transducer where blank symbols at the beginning and ending of the graph is penalized
    # Last resort: use a cheap alignment model to get a subgraph of the big graph first

    # blank_penalty should be negative

    fst_graph = k2.ctc_graph([word_id_list], modified=False, device='cpu')[0]

    c_str = k2.to_str_simple(fst_graph)
    arcs = c_str.strip().split("\n")
    arcs = [x.strip() for x in arcs if len(x.strip()) > 0]
    final_state = int(arcs[-1])
    
    arcs = arcs[:-1]
    arcs = [tuple(map(int, a.split())) for a in arcs]
    # ss, ee, l1, l2, w = arc

    arc0 = arcs[0]
    arcs_last = [a for a in arcs[-5:] if a[2] > 0]

    arcs = [(0, 0, 0, 0, blank_penalty)] + arcs[1:-5] + arcs_last

    non_eps_nodes = set((arc[1], arc[3]) for arc in arcs if arc[3] > 0)   # if this node has a non-eps in-coming arc
    arcs += [(0, n, l, l, 0) for n, l in non_eps_nodes if n > 1]

    # arcs += [(n, final_state, -1, -1, 0) for n in range(1, final_state - 2)]
    arcs += [(n, final_state - 1, 0, 0, blank_penalty) for n, l in non_eps_nodes]
    arcs += [(final_state - 1, final_state - 1, 0, 0, blank_penalty)]
    arcs += [(final_state - 1, final_state, -1, -1, 0)]

    new_arcs = arcs
    new_arcs.append([final_state])

    new_arcs = sorted(new_arcs, key=lambda arc: arc[0])
    new_arcs = [[str(i) for i in arc] for arc in new_arcs]
    new_arcs = [" ".join(arc) for arc in new_arcs]
    new_arcs = "\n".join(new_arcs)

    if return_str:
        return new_arcs
    else:
        fst = k2.Fsa.from_str(new_arcs, acceptor=False)
        return fst

In [14]:
libri_long_text = get_long_text([c], sp=sp, make_fst=True)
print(libri_long_text)
fst = list(libri_long_text.items())[0][1]
print(fst.shape[0], fst.num_arcs)

100%|██████████| 1/1 [00:00<00:00, 686.47it/s]

{(4362, 15663): <k2.fsa.Fsa object at 0x1553fa703e50>}
144 708


In [15]:
batch = {
    "supervisions": {
        "text": [c.supervisions[0].text],
        "cut": [c],
    },
    "inputs": torch.Tensor(c.load_features()).unsqueeze(0),
}

In [16]:
feature = batch["inputs"]
assert feature.ndim == 3

supervisions = batch["supervisions"]
feature_lens = torch.Tensor([feature.size(1)]).int()
print(feature_lens, feature.shape)

tensor([974], dtype=torch.int32) torch.Size([1, 974, 80])


In [17]:
# texts = batch["supervisions"]["text"]
# y = sp.encode(texts, out_type=int)
# y = k2.RaggedTensor(y)

cuts = batch['supervisions']['cut']
y_list = [libri_long_text[tuple(get_uid_key(c.id)[:2])] for c in cuts]
# y_list = [libri_long_text[(6415, 100596)] for c in cuts]  # A mismatched text

In [18]:
targets = y_list
x = feature
x_lens = feature_lens
prune_range = params.prune_range
am_scale = params.am_scale
lm_scale = params.lm_scale

In [19]:
encoder_out, encoder_out_lens = model.forward_encoder(x, x_lens)
ctc_output = model.ctc_output(encoder_out)

In [20]:
ctc_output.argmax(dim=-1)

tensor([[  0,   0,   0,   0, 141,   0,   0,   0,   0, 334, 236, 236,   0,   0,
           0,  14,   0,   0,  12,   0, 209,   0,   0,   0,   4,   4,   0, 489,
         489,   0,   0,   0,   0,   0,   0,   0,  10,  10,   0,   0,  83,   0,
           0,   0, 121,   0,   0,  71,   0,   0, 174,   0,   0,   0,  63,   0,
          14,   0, 273,   0,   0,   0,   0,   0, 200,   0,  42,   0,   6,   6,
           0,   0, 122,  16,  16,  20,   0,   0,  17,   0,   4,   0,   0,   0,
          67,   6,  16,  34,   0,  26,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   8,   0,   7,   0,   0,   0,  34,  15,   0,  69,  65,  65,
           0,  14,  14,   0,   4,   0,  61,  80,   0,   0, 446, 446,   0,   0,
           0,  10,  10,   0, 150,   0,   0,   0,  63,  14,  14,  14, 126,   0,
         123,   0,  34,  34,   3,  26,   0,   0,  50,   0,   0,   0, 146,   0,
           0,   0,  10,   0,   4,   0,   0, 338,  91,   0,   0,   0,  16,   0,
          26,   0,   0,   0,  83,   0,   0,   0, 382

In [21]:
input_lengths = encoder_out_lens
batch_size = input_lengths.size(0)
# TODO
supervision_segments = torch.stack(
    (
        torch.arange(batch_size),
        torch.zeros(batch_size),
        input_lengths.cpu(),
    ),
    1,
).to(torch.int32)
indices = torch.argsort(supervision_segments[:, 2], descending=True)
supervision_segments = supervision_segments[indices]

In [22]:
y_list = targets
_y_list = [y_list[i] for i in indices.tolist()]
decoding_graph = k2.create_fsa_vec(_y_list)
decoding_graph = k2.arc_sort(decoding_graph)
decoding_graph = decoding_graph.to(encoder_out.device)
decoding_graph.shape[0], decoding_graph.num_arcs

(1, 708)

In [23]:
log_probs = ctc_output
lattice = get_lattice(
    nnet_output=log_probs,
    decoding_graph=decoding_graph,
    supervision_segments=supervision_segments,
    search_beam=15,
    output_beam=6,
    min_active_states=30,
    max_active_states=10000,
    subsampling_factor=params.subsampling_factor,
)
lattice.shape[0], lattice.num_arcs

(1, 1902)

In [24]:
best_path = one_best_decoding(
    lattice=lattice,
    use_double_scores=True,
)

In [25]:
hyps = get_texts(best_path)
print(f"[ref]: {c.supervisions[0].text}")
print(f"[hyp]: {sp.decode(hyps)[0]}")

[ref]: SUGGESTING TO US THE IDEA OF AT ONCE SEEKING OUR SAFE CAMP IN THE TIMBER AND ABANDONING THE PURPOSE OF MAKING AN OBSERVATION OF THE BAROMETER AT THREE P M TWO HAVING ALREADY BEEN MADE
[hyp]: SUGGESTING TO US THE IDEA OF AT ONCE SEEKING OUR SAFE CAMP IN THE TIMBER AND ABANDONING THE PURPOSE OF MAKING AN OBSERVATION OF THE BAROMETER AT THREE P M TWO HAVING ALREADY BEEN MADE


In [26]:
# https://github.com/k2-fsa/icefall/tree/master/egs/librispeech/ASR/conformer_ctc
# https://github.com/k2-fsa/icefall/blob/master/icefall/ali.py
# https://github.com/k2-fsa/icefall/pull/1482
# https://github.com/k2-fsa/icefall/pull/876

labels_ali = get_alignments(best_path, kind="labels")
aux_labels_ali = get_alignments(best_path, kind="aux_labels")

In [28]:
decoding_results = get_texts_with_timestamp(best_path)

In [31]:
decoding_results.timestamps